Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the cells accordingly.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "江詠筑"
COLLABORATORS = ""

---

# Introduce to OpenAI Gym with DuckieNav Environment


We will introduce the main API methods in gym:
* `reset()`
* `step()`
* `render()`

![hi](https://drive.google.com/uc?id=193KoOaA5YPOha8TnxdHXDXgsciIcSuzV)

and the essentials in RL:
* Environment
* State
* Action
* Reward
* Agent

### Environment
The example is modifed from the Taxi Problem in "Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition" by  Tom Dietterich (2000), Journal of Artificial Intelligence Research.

We consider shows a 14 by 9 grid world, except the "service area." The taxi problem is episodic, and in each episode a passenger is located at one of the 4 specially designated locations (R, Y, B, and G). The taxi(agent) starts in a given location and must go to the transported passenger’s location, pick up the passenger, go to the destination location, and put down the passenger. The episode ends when the passenger is deposited at the destination location to one of the 4 locations.

```
MAP = [
    "+-----------------+",
    "|O|O| : : : : :G: |",
    "|O|O| |O| |O| |O| |",
    "|O| : |O| |O| |O| |",
    "| : |O|O| : : : : |",
    "| |O|O|O|O|O| |O| |",
    "| : :R: : : : |O| |",
    "| |O|O|O| |O|O|O| |",
    "| |O| : : |O| : : |",
    "| |O| |O|O|O|B|O| |",
    "| : : : : : : |O| |",
    "| |O| |O| |O| |O| |",
    "| : : : : : : |O| |",
    "| |O| |O| |O|O|O| |",
    "| : : :Y: : : : : |",
    "+-----------------+",
]
```

![hi](https://drive.google.com/uc?id=1DRQ5bLGDz_NC-uyuUoPpZRBiCBQgiqsl)
<br />




Adapted from https://www.oreilly.com/learning/introduction-to-reinforcement-learning-and-openai-gym


## 0. Understand pytest

In [ ]:
!python --version
!pytest --version

Python 3.7.12
This is pytest version 3.6.4, imported from /usr/local/lib/python2.7/dist-packages/pytest.pyc


In [ ]:
%%file test_math.py

import math
def test_add():
    assert 1+1 == 2

def test_mul():
    assert 6*7 == 42

def test_sin():
    assert math.sin(0) == 0

Writing test_math.py


In [ ]:
!python -m pytest test_math.py 

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-3.6.4, py-1.10.0, pluggy-0.7.1
rootdir: /content, inifile:
plugins: typeguard-2.7.1
collected 3 items                                                              

test_math.py ...                                                         [100%]

=========================== 3 passed in 0.03 seconds ===========================


## 0. DFS in Froken Lake

* S: initial state
* F: frozen lake
* H: hole
* G: the goal
* Red square: indicates the current position of the player

In [ ]:
import gym # loading the Gym library
from gym.envs.toy_text.frozen_lake import generate_random_map
 
env_fl = gym.make("FrozenLake-v0")
env_fl.reset()                    
env_fl.render()


SFFF
FHFH
FFFH
HFFG


### Part A (1 + 1 points)

Hint:
* You could take a look at https://github.com/openai/gym/blob/master/tests/envs/test_frozenlake_dfs.py
* You need to take care of the size of the map in the code.

In [ ]:
def frozenlake_dfs_path_exists(res):
    '''
    param res: a map (note: you need to take care of size of map in your solution)
    out_1: if the dfs path exists
    out_2: discovered
    '''
    frontier, discovered = [], set()
    frontier.append((0, 0))
    while frontier:
      r, c = frontier.pop()
      if not (r, c) in discovered:
        discovered.add((r, c))
        directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]        
        for x, y in directions:
          r_new = r + x
          c_new = c + y
          if r_new < 0 or r_new >= len(res) or c_new < 0 or c_new >= len(res[0]) :
            continue
          if res[r_new][c_new] == "G":
            return True,discovered
          if res[r_new][c_new] not in "#H":
            frontier.append((r_new, c_new))
    return False,discovered

    

In [ ]:
map_sizes = [5, 8]
for size in map_sizes:
    new_frozenlake = generate_random_map(size)
    assert len(new_frozenlake) == size
    assert len(new_frozenlake[0]) == size
    out_1, out_2 = frozenlake_dfs_path_exists(new_frozenlake)
    assert out_1 == True

In [ ]:
MAPS = {
    "4x4": [
        "SFFF", 
        "FHFH", 
        "FFFH", 
        "HFFG"],
}
out_1, out_2 = frozenlake_dfs_path_exists(MAPS["4x4"])
assert out_2 == {(0, 1), (1, 2), (2, 1), (0, 0), (3, 1), (0, 3), (2, 0), (0, 2), (2, 2), (1, 0), (3, 2)}

## 1. Initialize DuckieNav-v2

### Installation
You can obtain and install this customized gym environment (https://github.com/ARG-NCTU/gym-duckienav.git): 


In [ ]:
!git clone https://github.com/ARG-NCTU/gym-duckienav.git
%cd gym-duckienav
!pip install -e .

Cloning into 'gym-duckienav'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 104 (delta 4), reused 3 (delta 0), pack-reused 91
Receiving objects: 100% (104/104), 825.81 KiB | 30.58 MiB/s, done.
Resolving deltas: 100% (35/35), done.
/content/gym-duckienav
Obtaining file:///content/gym-duckienav
  Running setup.py develop for gym-duckienav


In [ ]:
import gym
import gym_duckienav
import gym.spaces
import numpy as np

env = gym.make("DuckieNav-v2")

### How many 'states' in observation_space: 
There are 2520 states from: 14 (rows) x 9 (columns) x 5 (passenger locations: R, Y, B, G, or on taxi) x 4 (destinations: R, Y, B, or G)

In [ ]:
env.observation_space.n

2520

### action_space: 
There are 6 possible actions in Taxi-v2 environment
* down (0), up (1), right (2), left (3), pick-up (4), and drop-off (5)

In [ ]:
env.action_space.n

6

## 2. States

Resets the state of the environment and returns an initial observation (state).

The current state is from :
* current taxi row position
* current taxi colum position
* passenger location (Blue or in taxi) from 0: R, 1: G, 2: Y, 3: B; 4: in taxi.
* destination location (Magenta) from 0: R, 1: G, 2: Y, 3: B

In [ ]:
env.reset()

print("Current state: ", str(env.s))
for p in env.decode(env.s): print(p)
env.render()

Current state:  161
0
8
0
1
+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : |O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+



Repeat previous cell for a few times.

In taxi problem, the colors mean:
* blue: passenger's current position
* magenta: destination
* yellow: empty taxi
* green: full taxi

## 3. Actions

Remember that the taxi agent can perform the following actions:
* 0: "South", 
* 1: "North", 
* 2: "East", 
* 3: "West", 
* 4: "Pickup", 
* 5: "Dropoff"

Let's set the state to 124.
Let the taxi agent perform some actions.  

In [ ]:
ACTIONS = ["South", "North", "East", "West", "Pickup", "Dropoff"]
env.s = 124
env.render()

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : |O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+



### `step()`

Run one timestep of the environment's dynamics. 
It returns a tuple (observation, reward, done, info)
* observation (object): agent's observation of the current environment
* reward (float) : amount of reward returned after previous action
* done (boolean): whether the episode has ended, in which case further step() calls will return undefined results
* info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)

Essentially the empty taxi is supposed to: 
* move toward the blue letter, 
* pickup the passenger (now the taxi is green), 
* drive to the magenta letter, and 
* drop the passenger (the taxi is yellow again).

It is obvious that we should start with moving "East" env.step(2). Index 2 is for moving "East"
We will do the followings:
* Perform "Pickup" step(4) (although the passenger is not here)
* Perform "East" step(2)
* Perform "Pickup" step(4)
* Perform "West" step(3)
* Perform "South" step(0) for 5 times
* Perfomr "Dropoff" (5)
* Perform "West" step(3) for 4 times
* Perfomr "Dropoff" (5)

In [ ]:
state, reward, done, info = env.step(4)
env.render()
print("reward: ", str(reward))

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : |O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (Pickup)
reward:  -10


In [ ]:
state, reward, done, info = env.step(2)
env.render()
print("reward: ", str(reward))

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : |O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (East)
reward:  -1


In [ ]:
state, reward, done, info = env.step(4)
env.render()
print("reward: ", str(reward))

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : |O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (Pickup)
reward:  -1


In [ ]:
state, reward, done, info = env.step(3)
env.render()
print("reward: ", str(reward))

+-----------------+
|O|O| : : : :_:G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : |O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (West)
reward:  -1


In [ ]:
for i in range(0, 5):
    env.step(0)
env.render()

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : :_|O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (South)


In [ ]:
state, reward, done, info = env.step(5)
env.render()
print("reward: ", str(reward))

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : :_|O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (Dropoff)
reward:  -10


In [ ]:
for i in range(0, 4):
    env.step(3)
env.render()

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : |O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (West)


In [ ]:
state, reward, done, info = env.step(5)
env.render()
print("reward: ", str(reward))
print("done: ", str(done))

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : |O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (Dropoff)
reward:  20
done:  True


### Rewards

You have probably figured out the rewards:
* Perform any movements: -1
* Pick up or drop off at the wrong position: -10
* Drop off the passenger at the right position: 20 

## 4. Random Agent: 

We will use the funciton env.action_space.sample(); you could run the following cell for a few times

In [ ]:
print(env.action_space.sample())

2


### How good does behaving completely random do?

In [ ]:
state = env.reset()

counter = 0
g = 0
reward = None
while reward != 20:
    state, reward, done, info = env.step(env.action_space.sample())
    counter += 1
    g += reward
print("Solved in {} Steps with a total reward of {}".format(counter,g))


Solved in 703 Steps with a total reward of -18103


You may luck out and solve the environment fairly quickly, but on average, a completely random policy will solve this environment in about ???? steps

## 5. Agent with Basic Reinforcement Learning: Q-Learning

In order to maximize our reward, we will have to have the algorithm remember its actions and their associated rewards. Here, the algorithm’s memory is going to be a Q action value table.

To manage this Q table, we will use a NumPy array. The size of this table will be the number of states (2520) by the number of possible actions (6).

In [ ]:
n_states = env.observation_space.n
n_actions = env.action_space.n
Q = np.zeros([n_states, n_actions])
ACTIONS = ["South", "North", "East", "West", "Pickup", "Dropoff"]

episodes = 1
G = 0
counter = 0
alpha = 0.618

for episode in range(1,episodes+1):
    done = False
    G, reward = 0,0

    state = env.reset()
    env.render()
    print("Step: {}, Action: {}, Reward: {}, Q[{}] \t{}".format("   ", "   ", "   ", "   ", ACTIONS))

    firstState = state
    print("Initial State = {}".format(state))
    while reward != 20:
        action = np.argmax(Q[state])  #1
        state2, reward, done, info = env.step(action) #2

        if counter < 20:
            print("Before Step: {}, Action: {}, Reward: {}, Q[{}] \t{}".format(counter, ACTIONS[action], reward, state, Q[state]))
            print("Future Step: {}, Action: {}, Reward: {}, Q[{}] \t{}".format(counter, ACTIONS[action], reward, state2, Q[state2]))

        Q[state,action] += alpha * (reward + np.max(Q[state2]) - Q[state,action]) #3
        
        if counter < 20:
            print("After  Step: {}, Action: {}, Reward: {}, Q[{}] \t{}".format(counter, ACTIONS[action], reward, state, Q[state]))
            print("--------")

        counter += 1
        G += reward
        state = state2
        
finalState = state
print("Final State = {}".format(finalState))
print("Solved in {} Steps with a total reward of {}".format(counter, G))
env.render()

print(ACTIONS)
print(Q[finalState])

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : |O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+

Step:    , Action:    , Reward:    , Q[   ] 	['South', 'North', 'East', 'West', 'Pickup', 'Dropoff']
Initial State = 109
Before Step: 0, Action: South, Reward: -50, Q[109] 	[0. 0. 0. 0. 0. 0.]
Future Step: 0, Action: South, Reward: -50, Q[289] 	[0. 0. 0. 0. 0. 0.]
After  Step: 0, Action: South, Reward: -50, Q[109] 	[-30.9   0.    0.    0.    0.    0. ]
--------
Before Step: 1, Action: South, Reward: -50, Q[289] 	[0. 0. 0. 0. 0. 0.]
Future Step: 1, Action: South, Reward: -50, Q[469] 	[0. 0. 0. 0. 0. 0.]
After  Step: 1, Action: South, Reward: -50, Q[289] 	[-30.9   0.    0.    0.    0.    0. ]
--------
Before Step: 2, Action: South, Reward: -1, Q[469] 	[0. 0. 0. 0. 0. 0.]
F

First (#1): The agent starts by choosing an action with the highest Q value for the current state using argmax. Argmax will return the index/action with the highest value for that state. Initially, our Q table will be all zeros. But, after every step, the Q values for state-action pairs will be updated.

Second (#2): The agent then takes action and we store the future state as state2 (S t+1). This will allow the agent to compare the previous state to the new state.

Third (#3): We update the state-action pair (St , At) for Q using the reward, and the max Q value for state2 (S t+1). This update is done using the action value formula (based upon the Bellman equation) and allows state-action pairs to be updated in a recursive fashion (based on future values). See the following Figure for the value iteration update.

![hi](https://drive.google.com/uc?id=1QqKZbyJagZ6laGb06dDFrj55zdjUj107)
<br />

In [ ]:
print(str(np.count_nonzero(Q)), " / ", str(n_states * n_actions))

2814  /  15120


### Let's run over multiple episodes so that we can converge on an optimal policy

In [ ]:
episodes = 10000
rewardTracker = []

G = 0
alpha = 0.618

for episode in range(1,episodes+1):
    done = False
    G, reward = 0,0

    state = env.reset()

    while done != True:
        action = np.argmax(Q[state]) 
        state2, reward, done, info = env.step(action) 
        Q[state,action] += alpha * ((reward + (np.max(Q[state2]))  - Q[state,action]))
        G += reward
        state = state2
    
    if episode % 100 == 0:
        print('Episode {} Total Reward: {}'.format(episode,G))
    
print(str(np.count_nonzero(Q)), " / ", str(n_states * n_actions))

Episode 100 Total Reward: -31
Episode 200 Total Reward: -2113
Episode 300 Total Reward: -109
Episode 400 Total Reward: -43
Episode 500 Total Reward: -135
Episode 600 Total Reward: -62
Episode 700 Total Reward: -293
Episode 800 Total Reward: -46
Episode 900 Total Reward: -10
Episode 1000 Total Reward: -47
Episode 1100 Total Reward: 5
Episode 1200 Total Reward: -45
Episode 1300 Total Reward: -67
Episode 1400 Total Reward: -60
Episode 1500 Total Reward: -49
Episode 1600 Total Reward: -42
Episode 1700 Total Reward: -42
Episode 1800 Total Reward: -112
Episode 1900 Total Reward: -81
Episode 2000 Total Reward: -53
Episode 2100 Total Reward: -41
Episode 2200 Total Reward: -56
Episode 2300 Total Reward: -46
Episode 2400 Total Reward: -49
Episode 2500 Total Reward: -47
Episode 2600 Total Reward: -56
Episode 2700 Total Reward: -106
Episode 2800 Total Reward: -161
Episode 2900 Total Reward: -53
Episode 3000 Total Reward: -46
Episode 3100 Total Reward: 9
Episode 3200 Total Reward: -59
Episode 3300 

### Now that we have learned the optimal Q Values we have developed an optimal policy and have no need to train the agent anymore

In [ ]:
state = env.reset()

done = None

while done != True:
    # We simply take the action with the highest Q Value
    action = np.argmax(Q[state])
    state, reward, done, info = env.step(action)
    env.render()

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : |O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (North)
+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : |O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (East)
+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : |O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (East)
+-----------

### Part B: 2 points

Hints:
* implement a function for the learned the optimal Q Values for a given state (124)
* You could use ACTIONS = ["South", "North", "East", "West", "Pickup", "Dropoff"] for out_1

In [ ]:
def test_q_learning(env , Q , state):
    '''
    out_1: the sequence of actions
    out_2: total reward
    out_3: total steps
    '''

    ACTIONS = ["South" , "North" , "East" , "West" , "Pickup" , "Dropoff"]
    action_seq = []
    tot_reward = 0
    tot_steps = 0
    done = None
    while done != True:
      action = np.argmax(Q[state])
      state , reward , done , info = env.step(action)
      action_seq.append(ACTIONS[action])
      tot_steps += 1
      tot_reward += reward
    return action_seq , tot_reward , tot_steps

In [ ]:

env.s = 124
env.render()
out_1 , out_2 , out_3 = test_q_learning(env , Q , env.s)
assert out_1 == ['East', 'Pickup', 'West', 'South', 'South', 'South', 'South', 'South', 'West', 'West', 'West', 'West', 'Dropoff']
assert out_2 == 8
assert out_3 == 13


+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : |O| |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (Dropoff)
